In [ ]:
import os
import platform
import mlflow
import urllib3
import logging

# Enable debug logging
logging.basicConfig(level=logging.DEBUG)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Your MLflow server URL
MLFLOW_TRACKING_URI = "https://ec2.....compute-1.amazonaws.com:XXXXX"  # Replace with your actual URI

# Configure MLflow with SSL verification disabled (for debugging only)
os.environ['MLFLOW_TRACKING_INSECURE_TLS'] = 'true'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Print configuration for debugging
print("Python version:", platform.python_version())
print("MLflow version:", mlflow.__version__)
print("Tracking URI:", mlflow.get_tracking_uri())

try:
    # Test the connection
    mlflow.set_experiment("youtube-sentiment-baseline")
    
    with mlflow.start_run():
        mlflow.log_param("param1", 15)
        mlflow.log_metric("metric1", 0.89)
        print("Successfully logged metrics and parameters")
        
except Exception as e:
    print("\nError occurred:")
    print(f"Type: {type(e).__name__}")
    print(f"Message: {str(e)}")
    print("\nTrying local tracking instead...")
    
    # Fall back to local tracking
    mlflow.set_tracking_uri("file:./mlruns")
    mlflow.set_experiment("youtube-sentiment-baseline")
    
    with mlflow.start_run():
        mlflow.log_param("param1", 15)
        mlflow.log_metric("metric1", 0.89)
        print("Successfully logged to local tracking server")
        print("Local tracking URI:", mlflow.get_tracking_uri())


In [1]:
import mlflow

In [5]:
mlflow.set_tracking_uri("http://ec2-3-80-217-32.compute-1.amazonaws.com:5000/")

with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.89)
    

2025/08/21 20:56:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run righteous-moose-147 at: http://ec2-3-80-217-32.compute-1.amazonaws.com:5000/#/experiments/0/runs/7809212c671e4d10a5e4d47edd7ac6df.
2025/08/21 20:56:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-3-80-217-32.compute-1.amazonaws.com:5000/#/experiments/0.


In [6]:
#creating baseline model

import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [8]:
df.dropna(inplace=True)

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df = df[~(df['clean_comment'].str.strip() == '')]

In [11]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [12]:
# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hadi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hadi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r'[^A-Za-z0-9\s!?.,]', '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

In [14]:
# Apply the preprocessing function to the 'clean_comment' column
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment)

In [15]:
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [16]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [17]:
# Step 1: Vectorize the comments using Bag of Words
vectorizer = CountVectorizer(max_features=10000)

In [18]:
X = vectorizer.fit_transform(df['clean_comment']).toarray()
y = df['category'] # Assuming 'category' is the target variable (0 or 1 for binary classification; but here -1, 0, 1 classes , right?)

In [19]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:
y

0        1
1        1
2       -1
3        0
4        1
        ..
37244    0
37245    1
37246    0
37247    1
37248    0
Name: category, Length: 36793, dtype: int64

In [21]:
X.shape

(36793, 10000)

In [24]:
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-3-80-217-32.compute-1.amazonaws.com:5000/")


In [25]:
# Set or create an experiment
mlflow.set_experiment("RF Baseline")

2025/08/21 21:08:25 INFO mlflow.tracking.fluent: Experiment with name 'RF Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-9999/191120065325554731', creation_time=1755803306332, experiment_id='191120065325554731', last_update_time=1755803306332, lifecycle_stage='active', name='RF Baseline', tags={}>

In [26]:
import boto3

In [27]:
import awscli

In [28]:
!aws configure

^C


In [ ]:
# Step 1: Split the data intro training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 2: Define and train a Random Forest baseline model
with mlflow.start_run() as run:
    # Log a description of the run
    mlflow.set_tag("mlflow.runName", "RandomForest_Baseline_TrainTestSplit")
    mlflow.set_tag("experiment_type", "baseline")
    mlflow.set_tag("model_type", "RandomForestClassifier")

    # Add a description
    mlflow.set_tag("description", "Baseline Random Forest model for sentiment analysis using Bag of Words (BoW)")

    # Log parameters of the vectorizer
    mlflow.log_param("vectorizer_type", "CountVectorizer")
    mlflow.log_param("vectorizer_max_features", vectorizer.max_features)

    # Log Random Forest parameters
    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Log metrics for each class and accruacy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    classification_rep = classification_report(y_test, y_pred, output_dict=True)

    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):  # For precision, recall, f1-score, etc.
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)
        
    # Confusion matrix plot
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")

    # Save and log the confusion matrix plot
    plt.savefig("confusion_matrix.png")
    mlflow.log_articat("/content/confusion_matrix.png")

    # Log the Random Forest model
    mlflow.sklearn.log_model(model, "random_forest_model")

    # Optionally log the dataset itself
    df.to_csv("dataset.csv", index=False)
    mlflow.log_artifact("/content/dataset.csv")


# Display final accuracy
print(f"Final accuracy: {accuracy}")
